In [1]:
import pinecone
import tqdm
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

pinecone.init(      
	api_key='f0b91127-5366-4d48-a42e-6a79f45e80f6',      
	environment='asia-northeast1-gcp'      
)      
index = pinecone.Index('skills-database')

import os
from dotenv import load_dotenv

load_dotenv("C:\\Users\\abuza\\Desktop\\jobd\\api.env")
API_KEY = os.getenv("API_KEY")
os.environ['OPENAI_API_KEY'] = API_KEY

c:\Users\abuza\anaconda3\envs\JDAIB\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [28]:
def read_and_separate_roles(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    
    # Split the data into individual roles based on two newline characters
    role = data.split('\n---')
    return role
def make_embeddings(string):
    embeddings = OpenAIEmbeddings()
    result = embeddings.embed_documents(texts=string)
    return result[0]

# Provide the file path to your text file
file_path = 'database.txt'

# Read and separate the roles
roles = read_and_separate_roles(file_path)
roles
# roles[2].split(':')[0]

['Data Science\n\n--Data Scientists Generalists: Good with programming (Python or R), data manipulation (e.g., pandas), basic SQL, statistics, business acumen, building basic ML algorithms and models, and basic data visualization (Tableau, etc.).\n\n--Data Scientists with Data Engineering Skills: Proficient with data management tools, ETL processes, Data pipeline creation and optimization, big data tools (e.g., Hadoop, Spark), and perhaps less knowledge of pure statistics and ML algorithms.\n\n--Data Scientists with Stakeholder Management: Strong business acumen, domain knowledge, stakeholder relationship management, ability to translate technical findings to nontechnical audiences, and potentially less hands-on with ML modeling and coding\n\n--Data Scientists Specializing in ML: Strong in ML and coding. Can build and possibly deploy ML models but may not be as focused on stakeholder management or business understanding. Proficiency in ML frameworks (e.g., scikit-learn, TensorFlow), Mo

In [33]:
i = 0
for role in roles:
    namespace = role.split("\n\n")[0]
    #print(namespace)
    n = role.split('--')[1:]
    print(n)
    # print(len(n))
    for subrole in n:
        role = subrole[i].split(':')[0]
        print(f'role: {role}')
        print (n)
        i = i+1
        break
    break

    #     index.upsert([
    #     (f'id-{i}', make_embeddings(role[i]), {'role_title': role, 'description': roles[i]})
    #     ], namespace=namespace)


    # # print (namespace)


['Data Scientists Generalists: Good with programming (Python or R), data manipulation (e.g., pandas), basic SQL, statistics, business acumen, building basic ML algorithms and models, and basic data visualization (Tableau, etc.).\n\n', 'Data Scientists with Data Engineering Skills: Proficient with data management tools, ETL processes, Data pipeline creation and optimization, big data tools (e.g., Hadoop, Spark), and perhaps less knowledge of pure statistics and ML algorithms.\n\n', 'Data Scientists with Stakeholder Management: Strong business acumen, domain knowledge, stakeholder relationship management, ability to translate technical findings to nontechnical audiences, and potentially less hands-on with ML modeling and coding\n\n', 'Data Scientists Specializing in ML: Strong in ML and coding. Can build and possibly deploy ML models but may not be as focused on stakeholder management or business understanding. Proficiency in ML frameworks (e.g., scikit-learn, TensorFlow), Model evaluati